In [ ]:
import sys
sys.path.append("..")

import pandas as pd
import json
from utils.busca_ceps import get_coords


In [ ]:
# Pega o CEP de todas as escolas
df_raw = pd.read_csv("../dados/temporarios/02-md_edb_limpo.csv")
cep_escolas = df_raw["CO_CEP"]
print(f"Numero escolas: {len(cep_escolas)}")  # 37 mil escolas


# Corrige os CEPs sem o zero a esquerda, transforma em str
cep_escolas = ["0"*(8-len(str(cep))) + str(cep) for cep in cep_escolas]
cep_escolas = set(cep_escolas)  # remove os repetidos
print(f"CEPs diferentes: {len(cep_escolas)}")  # 29 mil CEPs

In [ ]:
# Baixa as coords do banco
with open("../dados/banco_dados/cep_coords.json", "r", encoding="utf-8") as f:
    banco_coords = json.load(f)

print(f"Já possuimos {len(banco_coords)} coords no banco")


# Separa apenas os CEPs que ainda nao temos no banco
ceps_baixar = [cep for cep in cep_escolas if str(cep) not in banco_coords]

print(f"Iremos baixar {len(ceps_baixar)} novas coords")


In [ ]:


# Baixa as novas coords da API
novas_coords = await get_coords(ceps_baixar)

print(f"Consegui baixar {len(novas_coords)} novas coords")

In [ ]:
# Junta coords novas e antigas e salva no banco
banco_coords.update(novas_coords)

with open("../dados/banco_dados/cep_coords.json", "w", encoding="utf-8") as f:
    json.dump(banco_coords, f, ensure_ascii=False, indent=4)

print(f"Agora o banco tem {len(banco_coords)} coords")